## The Riemann solver for the Euler Equations
<img src="images/image copy.png" width="600">
<img src="images/image.png" width="600">

In [1]:
import numpy as np

In [ ]:
# The Riemann solver for the Euler Equations

TOL = 1e-6
#γ는 비열비
GAMMA = 1.4

def compute_f_and_df(p, p_l, p_r, rho_l, rho_r, al, ar, Al, Ar, Bl, Br):
    '''

    '''
    #left shock wave
    if(p > p_l):
        fl = (p - p_l)* np.sqrt(Al / (p + Bl))
        d_fl = np.sqrt(Al / (Bl + p)) * (1 - 0.5 * (p - p_l) / (Bl + p))
    #left rarefaction wave
    else:
        fl = 2 * al / (GAMMA - 1) * ((p/p_l)**((GAMMA - 1)/(2 * GAMMA)) - 1)
        d_fl = 1 / rho_l / al * (p/p_l)**(-(GAMMA + 1)/(2 * GAMMA))

    #right shock wave
    if(p > p_r):
        fr = (p - p_r)* np.sqrt(Ar / (p + Br))
        d_fr = np.sqrt(Ar / (Br + p)) * (1 - 0.5 * (p - p_r) / (Br + p))
    #right rarefaction wave
    else:
        fr = 2 * ar / (GAMMA - 1) * ((p/p_r)**((GAMMA - 1)/(2 * GAMMA)) - 1)
        d_fr = 1 / rho_r / ar * (p/p_r)**(-(GAMMA + 1)/(2 * GAMMA))

    return fl, d_fl, fr, d_fr

def solve_riemann_star_state(rho_l, u_l, p_l, rho_r, u_r, p_r):
    """
    Get the exact Riemann solution for the Euler equations.
    Newton-Raphson iterative procedure

    p(k)= p(k-1)- f(p(k-1)) / f'(p(k-1))

    return value: p*, u*, rho*l, rho*r
    """

    loop_count = 0

    Al = 2 / (GAMMA + 1) / rho_l
    Ar = 2 / (GAMMA + 1) / rho_r
    Bl = (GAMMA - 1) / (GAMMA + 1) * p_l
    Br = (GAMMA - 1) / (GAMMA + 1) * p_r
    al = np.sqrt(GAMMA * p_l / rho_l)
    ar = np.sqrt(GAMMA * p_r / rho_r)
    print(al + ar)

    # Initial guess for the pressure
    # Should be optimaized using Two–Rarefaction approximation, primitive variables, Two–Shock approximation.
    p = 0.5 * (p_l + p_r)
    while(True):
        loop_count += 1
        prev_p = p
        fl, d_fl, fr, d_fr = compute_f_and_df(p, p_l, p_r, rho_l, rho_r, al, ar, Al, Ar, Bl, Br)

        f = fl + fr  + u_r - u_l
        df = d_fl + d_fr

        p = p - f / df
        #음압 방지.
        p = max(p, 1e-12)
        if(2 * abs(p - prev_p) < TOL * (p + prev_p) or loop_count > 1000):
            break
    
    fl, d_fl, fr, d_fr = compute_f_and_df(p, p_l, p_r, rho_l, rho_r, al, ar, Al, Ar, Bl, Br)
    u = 0.5 * (u_l + u_r + fr - fl)
    if(p > p_l):
        rho_l_star = rho_l * (GAMMA * (p + p_l) - p_l + p) / (GAMMA * (p + p_l) - p + p_l)
    else:
        rho_l_star = rho_l * (p / p_l)**(1 / GAMMA)

    if(p > p_r):
        rho_r_star = rho_r * (GAMMA * (p + p_r) - p_r + p) / (GAMMA * (p + p_r) - p + p_r)
    else:
        rho_r_star = rho_r * (p / p_r)**(1 / GAMMA)

    print(loop_count)
    return p, u, rho_l_star, rho_r_star
    
    

<img src="images/image copy 2.png" width="600">


In [ ]:
# on s = x/t given, return the value of p, u, rho.

def evaluate_riemann_solution(s, rho_l, u_l, p_l, rho_r, u_r, p_r):
    p_star, u_star, rho_l_star, rho_r_star = solve_riemann_star_state(rho_l, u_l, p_l, rho_r, u_r, p_r)
    #Left Side of Contact
    if(s < u_star):
        #left rarefraction wave
        if(p_star < p_l):
            #speed of sound in left state
            al = np.sqrt(GAMMA * p_l / rho_l)
            #speed of head of left rarefaction wave
            s_hl = u_l - al
            if(s < s_hl):
                return p_l, u_l, rho_l
            al_star = al * (p_star / p_l)**((GAMMA - 1)/(2 * GAMMA))
            s_tl = u_star - al_star
            if(s > s_tl):
                return p_star, u_star, rho_l_star
            p = p_l * ((2 * al + (GAMMA - 1) * (u_l - s)) / (al * (GAMMA + 1)))**(2 * GAMMA / (GAMMA - 1))
            u = 2 / (GAMMA + 1) * (al + (GAMMA - 1) / 2 * u_l + s)
            rho = rho_l * ((2 * al + (GAMMA - 1) * (u_l - s)) / (al * (GAMMA + 1)))**(2 / (GAMMA - 1))
            return p, u, rho
        #left shock wave
        else:
            al = np.sqrt(GAMMA * p_l / rho_l)
            #speed of left shock wave
            s_l = u_l - al * np.sqrt((GAMMA * (p_star + p_l) + p_star - p_l) / (2 * GAMMA * p_l))
            if(s < s_l):
                return p_l, u_l, rho_l
            return p_star, u_star, rho_l_star
    #Right Side of Contact
    else:
        #right rarefraction wave
        if(p_star < p_r):
            #speed of sound in right state
            ar = np.sqrt(GAMMA * p_r / rho_r)
            #speed of head of right rarefaction wave
            s_hr = u_r + ar
            if(s > s_hr):
                return p_r, u_r, rho_r
            ar_star = ar * (p_star / p_r)**((GAMMA - 1)/(2 * GAMMA))
            s_tr = u_star + ar_star
            if(s > s_tr):
                return p_star, u_star, rho_r_star
            p = p_r * ((2 * ar + (GAMMA - 1) * (u_r - s)) / (ar * (GAMMA + 1)))**(2 * GAMMA / (GAMMA - 1))
            u = 2 / (GAMMA + 1) * (ar + (GAMMA - 1) / 2 * u_r + s)
            rho = rho_r * ((2 * ar + (GAMMA - 1) * (u_r - s)) / (ar * (GAMMA + 1)))**(2 / (GAMMA - 1))
            return p, u, rho
        #right shock wave
        else:
            ar = np.sqrt(GAMMA * p_r / rho_r)
            s_r = u_r + ar * np.sqrt((GAMMA * (p_star + p_r) + p_star - p_r) / (2 * GAMMA * p_r))
            if(s > s_r):
                return p_r, u_r, rho_r
            return p_star, u_star, rho_r_star
            
            

In [ ]:
#check if vacuum state is generated.
#true -> vacuum state is generated.
#false -> vacuum state is not generated.

def check_vacuum_state(rho_l, u_l, p_l, rho_r, u_r, p_r):
    al = np.sqrt(GAMMA * p_l / rho_l)
    ar = np.sqrt(GAMMA * p_r / rho_r)
    return 2 * (al + ar) / (GAMMA - 1) < u_r - u_l


In [4]:
p, u, rho_l_star, rho_r_star = solve_riemann_star_state(1.0, -2.0, 0.4, 1.0, 2.0, 0.4)
print(p, u, rho_l_star, rho_r_star)

1.4966629547095764
13
0.001893873420054761 0.0 0.02185211820681281 0.02185211820681281
